# How to choose an inference method

`sbi` implements three sets of methods for inferring the posterior:
- `NPE` (Neural Posterior Estimation),
- `NLE` (Neural Likelihood Estimation), and
- `NRE` (Neural Ratio Estimation).

For each of these methods, `sbi` implements an amortized version and a sequential version. Which algorithm should you choose?

## Explicit recommendations

- If have high-dimensional simulation-outputs (images, time-series,...), use `NPE` or `NRE`
- If you want fast sampling, use `NPE`
- If you have iid observations across many trials (and simulations are relatively expensive), use `NRE` or `NLE`
- If you want to perform inference for many observations, use amortized methods (which is the default of the `sbi` package).
- If you want to perform inference for few observations (<10), then start with amortized methods and switch to sequential methods only if the simulation cost is too high.
- Do not use `SNPE_A` or `SNPE_B` unless you are an expert user and know what you are doing.

## Additional explanation

### NPE vs NLE vs NRE

**`NPE`** is the only method which estimates the posterior directly. This means that it will not require further sampling steps (such as MCMC) _after_ training. This makes NPE the fastest to `.sample()` from the posterior estimate after training. In addition, it can use an embedding network to learn summary statistics (just like NRE, but unlike NLE), see the tutorial [here]().

**`NLE`** learns the likelihood. This allows NLE to emulate the simulator after training. To sample from the posterior, NLE is combined with MCMC or variational inference (see [here]()), which makes it slower to `.sample()`. An advantage of NLE is that, if you have multiple iid observations, then it can be much more simulation efficient than NPE (see tutorial [here]()).

**`NRE`** learns the likelihood-to-evidence ratio. It trains only a classifier (unlike NPE and NLE, which train a generative model), which makes it fastest to train. Like NLE, NRE has to be combined MCMC or variational inference to run `.sample()`. Like NPE, NRE can use an embedding network to learn summary statistics.

### Amortized or sequential

**Amortized** methods allow you to perform inference for _any_ observation without running new simulations or retraining. In contrast, **sequential** methods tailor inference to a single observation, which can make them more simulation efficient.

If you want to perform inference for many observations (>10), use amortized methods. If you have only few observations (or just a single on), then we still recommend you start with amortized methods. Only if simulation cost becomes too large, switch to sequential methods.

The default behavior of `sbi` is to run each method in its amortized form. To run sequential methods, you have to build follow the tutorial [here](https://sbi.readthedocs.io/en/latest/advanced_tutorials/02_multiround_inference.html).